# MLP Validation on NIPS4Bplus

In [1]:
import os
import numpy as np
import pandas as pd

import torch

from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn import preprocessing

from bring_features_and_file_paths import bring_features_and_file_paths
from bring_features_and_file_paths import from_spectrogram_path_to_BirdNET_output_path
from bring_features_and_file_paths import get_BirdNET_detections
from bring_features_and_file_paths import get_spectrogram_time_mark_in_file

In [2]:
feats, fpaths = bring_features_and_file_paths('/grand/projects/BirdAudio/Soundscapes/NIPS4Bplus/Features/', sub_sample=None)

We have 6703 feature vectors.


In [3]:
bird_species_df = pd.read_csv ('/grand/projects/BirdAudio/Soundscapes/NIPS4Bplus/nips4b_birdchallenge_espece_list.csv')
bird_species_df

,class number,class name,English_name,Scientific_name,type
0,none,Empty,Empty,noise sample,noise
1,1,Aegcau_call,Long-tailed Tit,Aegithalos caudatus,bird
2,2,Alaarv_song,Eurasian Skylark,Alauda arvensis,bird
3,3,Anttri_song,Tree Pipit,Anthus trivialis,bird
4,4,Butbut_call,Common Buzzard,Buteo buteo,bird
...,...,...,...,...,...
83,83,Trotro_song,Winter Wren,Troglodytes troglodytes,bird
84,84,Turmer_call,Common Blackbird,Turdus merula,bird
85,85,Turmer_song,Common Blackbird,Turdus merula,bird
86,86,Turphi_call,Song Thrush,Turdus philomelos,bird


In [4]:
bird_labels_df = pd.read_csv ('/grand/projects/BirdAudio/Soundscapes/NIPS4Bplus/nips4b_birdchallenge_train_labels.csv')
bird_labels_df=bird_labels_df.iloc[2:]
bird_labels_df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90
2,nips4b_birds_trainfile001.wav,3,NaN,­,NaN,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nips4b_birds_trainfile002.wav,1,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nips4b_birds_trainfile003.wav,2,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,nips4b_birds_trainfile004.wav,3,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
6,nips4b_birds_trainfile005.wav,3,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,nips4b_birds_trainfile684.wav,1,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686,nips4b_birds_trainfile685.wav,2,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
687,nips4b_birds_trainfile686.wav,3,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688,nips4b_birds_trainfile687.wav,3,NaN,­,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
vectors=[]
labels=[]
aux_labels=[]
for k, (fpath, feat) in enumerate(tqdm(zip(fpaths,feats), total=len(feats))):
    file_path = ''.join([chr(int(x)) for x in fpath]).replace('~','')
    if file_path.split('/')[3]=='train':
        Filename=os.path.basename(file_path[:-6]+'.wav')
        no_bird=True
        for k1, column in enumerate(bird_labels_df[bird_labels_df['Unnamed: 0']==Filename].iteritems()):
            if k1 > 3:
                status=np.array(column[1])[0]
                #print(status)
                if status=='1':
                    aux=bird_species_df[bird_species_df['class number']==str(k1-3)]
                    aux=aux['class name']
#                     print(k1-3)
#                     print(aux[k1-3].replace('_', ' '))
                    no_bird=False
                    vectors.append(np.array(feat))
                    aux_labels.append(aux[k1-3].replace('_', ' '))
                    labels.append(k1-3)
                    
        if no_bird:
            vectors.append(np.array(feat))
            aux_labels.append(' NONE ')
            labels.append(0)
#             print(0)
#             print(' NONE ')


vectors=np.array(vectors)
labels=np.array(labels)

unique_labels=np.unique(aux_labels)

 45%|████▌     | 3041/6703 [00:00<00:00, 9998.11it/s] /tmp/ipykernel_4318/2046338983.py:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for k1, column in enumerate(bird_labels_df[bird_labels_df['Unnamed: 0']==Filename].iteritems()):
100%|██████████| 6703/6703 [00:11<00:00, 560.49it/s] 


In [6]:
vectors.shape

(5258, 384)

In [7]:
labels.shape

(5258,)

In [8]:
unique_labels

array([' NONE ', 'Aegcau call', 'Alaarv song', 'Anttri song',
       'Butbut call', 'Carcan call', 'Carcan song', 'Carcar call',
       'Carcar song', 'Cerbra call', 'Cerbra song', 'Cetcet song',
       'Chlchl call', 'Cicatr song', 'Cicorn song', 'Cisjun song',
       'Colpal song', 'Corcor call', 'Denmaj call', 'Denmaj drum',
       'Embcir call', 'Embcir song', 'Erirub call', 'Erirub song',
       'Fricoe call', 'Fricoe song', 'Galcri call', 'Galcri song',
       'Galthe call', 'Galthe song', 'Gargla call', 'Hirrus call',
       'Jyntor song', 'Lopcri call', 'Loxcur call', 'Lularb song',
       'Lusmeg call', 'Lusmeg song', 'Lyrple song', 'Motcin call',
       'Musstr call', 'Oriori call', 'Oriori song', 'Parate call',
       'Parate song', 'Parcae call', 'Parcae song', 'Parmaj call',
       'Parmaj song', 'Pasdom call', 'Pelgra call', 'Petpet call',
       'Petpet song', 'Phofem song', 'Phycol call', 'Phycol song',
       'Picpic call', 'Plaaff song', 'Plasab song', 'Poepal call',


In [9]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,), random_state=1, activation='identity', max_iter=2000)

In [10]:
kf = KFold(n_splits=10)

In [11]:
performances=[]
for train_indices, test_indices in tqdm(kf.split(vectors), total=10):
    scaler = preprocessing.StandardScaler().fit(vectors[train_indices])
    clf.fit(scaler.transform(vectors[train_indices]), labels[train_indices])
    performances.append(clf.score(scaler.transform(vectors[test_indices]), labels[test_indices]))

  0%|          | 0/10 [00:00<?, ?it/s]/home/demattie/.conda/envs/soundscape_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
 10%|█         | 1/10 [01:31<13:43, 91.54s/it]/home/demattie/.conda/envs/soundscape_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
 20%|██        | 2/10 [02:58<11:50, 

In [12]:
np.mean(np.array(performances))

0.22517653449212385

In [13]:
# mask=np.array([unique_labels==' NONE '])
# mask=np.logical_or(mask,np.array([unique_labels=='Aegcau call']))
# mask=np.logical_or(mask,np.array([unique_labels=='Alaarv song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Butbut call']))
mask=np.array([unique_labels=='Carcan song'])
# mask=np.logical_or(mask,np.array([unique_labels=='Carcan song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Carcar song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Cetcet song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Cicatr song']))
mask=np.logical_or(mask,np.array([unique_labels=='Cisjun song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Embcir song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Erirub call']))
# mask=np.logical_or(mask,np.array([unique_labels=='Erirub song']))
mask=np.logical_or(mask,np.array([unique_labels=='Galcri call']))
# mask=np.logical_or(mask,np.array([unique_labels=='Galthe call']))
mask=np.logical_or(mask,np.array([unique_labels=='Lularb song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Lyrple song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Oriori call']))
mask=np.logical_or(mask,np.array([unique_labels=='Parcae song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Pelgra call']))
mask=np.logical_or(mask,np.array([unique_labels=='Petpet call']))
# mask=np.logical_or(mask,np.array([unique_labels=='Petpet song']))
mask=np.logical_or(mask,np.array([unique_labels=='Phycol song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Serser song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Siteur call']))
mask=np.logical_or(mask,np.array([unique_labels=='Strtur song']))
# mask=np.logical_or(mask,np.array([unique_labels=='Sylatr call']))
# mask=np.logical_or(mask,np.array([unique_labels=='Sylcan call']))
# mask=np.logical_or(mask,np.array([unique_labels=='Sylund call']))
mask=np.logical_or(mask,np.array([unique_labels=='Sylund song']))
mask=np.logical_or(mask,np.array([unique_labels=='Tetpyg song']))
mask=np.logical_or(mask,np.array([unique_labels=='Turphi call']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))
#     mask=np.logical_or(mask,np.array([unique_labels=='No detection']))

chosen_labels=np.where(mask[0])[0]
chosen_labels

def find_indices(A, B):
    indices = np.where(np.isin(A, B))
    return indices

indices=find_indices(labels, chosen_labels)[0]

vectors = vectors[indices]
labels = labels[indices]


In [14]:
performances=[]
for train_indices, test_indices in tqdm(kf.split(vectors), total=10):
    scaler = preprocessing.StandardScaler().fit(vectors[train_indices])
    clf.fit(scaler.transform(vectors[train_indices]), labels[train_indices])
    performances.append(clf.score(scaler.transform(vectors[test_indices]), labels[test_indices]))

100%|██████████| 10/10 [00:35<00:00,  3.58s/it]


In [15]:
np.mean(np.array(performances))

0.8747519841269842